In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pyshorteners
import pandas as pd
import numpy as np
import nltk
from newspaper import Article, ArticleException
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/kunal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
link = "https://news.google.com/"
url_shortener = pyshorteners.Shortener()

In [3]:
html_doc = requests.get('https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen')
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [4]:
boxes = soup.findAll('div', {'jslog': '88374'})

In [5]:
box = boxes[0].find('article').find('a').get('href')
article = Article(link+box)
article.download()
article.parse()
article.nlp()
print(article.publish_date)

None


In [6]:
text_map = dict()

In [7]:
first = boxes[0].findAll('article')[0].find('a').get('href')
second = boxes[1].findAll('article')[0].find('a').get('href')
print(link + first)
print(link + second)

https://news.google.com/./articles/CAIiEJSqetnyQTEkzsiOTgv09XkqGQgEKhAIACoHCAowsLXdCjCm3dEBMOThpAM?hl=en-IN&gl=IN&ceid=IN%3Aen
https://news.google.com/./articles/CAIiEJt7Wq9C7oBR6QWdY-MKvcEqGQgEKhAIACoHCAowzrL9CjDC7vQCMK2y1gU?hl=en-IN&gl=IN&ceid=IN%3Aen


In [8]:
main_news_data = []
for i, box in enumerate(boxes):
    main_news_temp = []
    articles = box.findAll('article')
    main_news_link = link + articles[0].find('a').get('href')
    url = url_shortener.tinyurl.short(main_news_link)
    main_news = Article(url)
    while True:
        try:
            main_news.download()
            break
        except ArticleException:
            pass
    main_news.parse()
    text = main_news.text
    lines = text.split('.')
    text_map[url] = lines
    main_news.nlp()
    main_news_temp.append(main_news.title)
    main_news_temp.append(main_news.summary)
    main_news_temp.append(main_news.publish_date)
    main_news_temp.append(url)
    main_news_data.append(main_news_temp)

In [12]:
main_news_df = pd.DataFrame(main_news_data, columns=['Title', 'Summary', 'Publish Date', 'Link'])

In [10]:
print(main_news_df.head(5))

                                               Title  \
0  Biggest crash ever makes India worst performin...   
1  Govt arms itself with powers to raise excise o...   
2  Airtel Urges Telecom Operators to Launch Intra...   
3  Gold price slips to a little below Rs 41K per ...   
4  Coronavirus Outbreak: Narendra Modi asks corpo...   

                                             Summary Publish Date  \
0  The Nifty dropped 1,135 points, or 13 per cent...         None   
1  Representative imageDownload The Times of Indi...         None   
2  To ensure seamless connectivity, Bharti Airtel...         None   
3  Gold price declined Rs 616 to Rs 40,719 per 10...         None   
4  New Delhi: Prime Minister Narendra Modi on Mon...         None   

                          Link  
0   http://tinyurl.com/s4ccecc  
1   http://tinyurl.com/rhlk9u2  
2   http://tinyurl.com/tpngpdr  
3   http://tinyurl.com/vpo4qxg  
4  http://tinyurl.com/yx5jsr2c  


In [11]:
sub_news_data = []
for i, box in enumerate(boxes):
    sub_news_temp = []
    articles = box.findAll('article')
    sub_news_link = link + articles[1].find('a').get('href')
    url = url_shortener.tinyurl.short(sub_news_link)
    sub_news = Article(url)
    while True:
        try:
            sub_news.download()
            break
        except ArticleException:
            pass
    sub_news.parse()
    text = sub_news.text
    lines = text.split('.')
    text_map[url] = lines
    sub_news.nlp()
    sub_news_temp.append(sub_news.title)
    sub_news_temp.append(sub_news.summary)
    sub_news_temp.append(sub_news.publish_date)
    sub_news_temp.append(url)
    sub_news_data.append(sub_news_temp)

KeyboardInterrupt: 

In [ ]:
sub_news_df = pd.DataFrame(sub_news_data, columns=['Title', 'Summary', 'Publish Date', 'Link'])

In [ ]:
print(sub_news_df.head(5))

In [ ]:
main_news_links = main_news_df[:]['Link'].values
sub_news_links = sub_news_df[:]['Link'].values
links = main_news_links + sub_news_links

In [ ]:
# memory efficient as not storing text but time inefficient as downloading each article and then parsing.
def search_word_space_efficient(word):
    for link in links:
        article = Article(link)
        article.download()
        article.parse()
        text = article.text
        lines = text.split('.')
        for i, line in enumerate(lines):
            if word in line:
                print("Found at: " + link)
                print(line + "\n")

In [ ]:
def search_word_time_efficient(word):
    for key, value in text_map.items():
        for line in value:
            if word in line:
                print("Found at: " + key)
                print(line + "\n")

In [ ]:
search_word_space_efficient("COVID-19")

In [ ]:
search_word_time_efficient("COVID-19")